<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-01-10 12:41:57
-------------------
qualified stocks: 89
with latest results: 84
still star stocks: 44
-------------------
Initial Investment:  1.04 C
CY Investment:  1.17 C
Reserve:  0.00
Current:  1.17 C
-------------------
Today PnL: -1.17 L (-0.99%)
Current PnL: -11.42 L (-9.76%)
CY Booked + Current PnL: 8.07 K (0.07%)
-------------------
Total profit:  4.39 L
Total loss:  -15.81 L
-------------------
Total Booked + Current PnL: 13.19 L (12.69%)
Total Booked PnL: 24.61 L (23.68%)
Curr Year Booked PnL: 11.50 L (9.8%)
Prev Year Booked PnL: 13.11 L (12.62%)
Est FTT:  1.87 C
Est FTT PnL: 69.72 L (59.41%)
-------------------
Est LTT:  2.48 C
Est LTT PnL: 1.31 C (111.4%)
Deployed:  1.04 C
Current:  1.17 C
CAGR/XIRR %: 10.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKBARODA,195.74,283.0,-27.43,M-LC,17.98,89945.0,12628.0,21839.0,-1.44,16.33,24.28,44.58,19.0,0.58,0.77,5.25,XR,NTT,BANKS
15,BAYERCROP,4346.57,6805.0,17.04,H-SC,22.47,86805.0,17260.0,22075.0,-2.78,24.82,25.43,56.56,21.0,0.78,0.74,11.13,X40N,NTT,CHEMICALS
48,LALPATHLAB,2630.06,3545.0,-15.47,H-SC,47.19,128304.0,12581.0,27675.0,-2.94,10.87,21.57,34.79,19.0,0.45,1.09,45.86,X40N,NTT,HEALTHCARE
80,TCS,4164.02,4998.0,5.26,X-LC,69.66,166803.0,4406.0,28123.0,5.90,2.71,16.86,20.03,12.0,0.16,1.42,17.06,X40,BTT,IT
72,SIS,477.00,528.0,63.14,M-SC,28.09,58760.0,-22330.0,31002.0,-0.32,-27.54,52.76,10.69,-21.0,-0.72,0.50,0.00,X40N,NTT,MISC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,4164.02,4998.00,5.26,X-LC,69.66,166803.0,4406.0,28123.0,5.90,2.71,16.86,20.03,12.0,0.16,1.42,17.06,X40,BTT,IT
50,LTIM,5664.39,7262.39,16.31,X-LC,53.93,183998.0,14066.0,33874.0,5.01,8.28,18.41,28.21,0.0,0.42,1.57,35.16,X200,ATH,IT
1,AARTIIND,487.04,919.00,-1.93,M-SC,68.54,71894.0,-9442.0,81578.0,4.14,-11.61,113.47,88.69,-13.0,-0.12,0.61,8.21,XR,NTT,CHEMICALS
42,IRCTC,904.87,1269.00,-23.89,M-MC,61.80,122975.0,-18185.0,74990.0,3.21,-12.88,60.98,40.24,12.0,-0.24,1.05,3.21,XY24,BTT,TRAVEL
41,INFY,1436.24,2275.00,9.90,X-LC,88.76,204849.0,55480.0,31752.0,2.73,37.14,15.50,58.40,37.0,1.75,1.75,42.94,X40,BTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,369.21,485.00,-63.24,M-MC,75.28,142025.0,-33350.0,88354.0,-7.56,-19.02,62.21,31.36,-13.0,-0.38,1.21,2.94,XY24,NTT,FMCG
78,TATAELXSI,7332.28,9000.00,-23.04,X-MC,2.25,84027.0,-18625.0,41971.0,-6.80,-18.14,49.95,22.74,-5.0,-0.44,0.72,0.00,X40N,NTT,IT
4,ALKYLAMINE,2347.98,4567.52,-9.26,X-SC,30.34,76256.0,-24707.0,120149.0,-5.18,-24.47,157.56,94.53,-61.0,-0.21,0.65,3.06,SR,ATH,CHEMICALS
81,TITAGARH,1117.46,1548.00,-5.31,M-SC,10.11,170474.0,-11672.0,81845.0,-4.16,-6.41,48.01,38.53,14.0,-0.14,1.45,27.02,XY24,NTT,ENGINEERING
29,GICRE,335.06,583.00,13.56,M-MC,97.75,194836.0,48415.0,59932.0,-4.10,33.07,30.76,74.00,137.0,0.81,1.66,50.60,XY24,BTT,INSURANCE


In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,33.76,M-SC,14.61,168687.0,31109.0,63308.0,-1.67,22.61,37.53,68.62,22.0,0.49,1.44,65.43,X40N,NTT,TEXTILES
15,BAYERCROP,4346.57,6805.00,17.04,H-SC,22.47,86805.0,17260.0,22075.0,-2.78,24.82,25.43,56.56,21.0,0.78,0.74,11.13,X40N,NTT,CHEMICALS
23,COLPAL,2837.00,3808.06,16.34,X-MC,82.02,142335.0,485.0,48067.0,-1.52,0.34,33.77,34.23,-54.0,0.01,1.21,18.10,X40,ATH,FMCG
50,LTIM,5664.39,7262.39,16.31,X-LC,53.93,183998.0,14066.0,33874.0,5.01,8.28,18.41,28.21,0.0,0.42,1.57,35.16,X200,ATH,IT
83,TTKPRESTIG,769.29,1149.24,15.65,X-SC,31.46,103313.0,2536.0,47235.0,-1.23,2.52,45.72,49.39,6.0,0.05,0.88,18.67,X40N,ATH,DURABLES


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Criteria'].isin(['XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,IEX,133.39,219.00,-7.01,H-SC,49.44,192043.0,42780.0,53023.0,-0.82,28.66,27.61,64.18,28.0,0.81,1.64,30.20,XR,NTT,MISC
12,BANKBARODA,195.74,283.00,-27.43,M-LC,17.98,89945.0,12628.0,21839.0,-1.44,16.33,24.28,44.58,19.0,0.58,0.77,5.25,XR,NTT,BANKS
54,METROPOLIS,1399.27,3102.00,13.76,X-SC,29.21,131266.0,37515.0,76567.0,-2.05,40.02,58.33,121.69,33.0,0.49,1.12,30.28,XR,NTT,HEALTHCARE
50,LTIM,5664.39,7262.39,16.31,X-LC,53.93,183998.0,14066.0,33874.0,5.01,8.28,18.41,28.21,0.0,0.42,1.57,35.16,X200,ATH,IT
30,GLAND,1463.94,3442.00,5.45,H-SC,89.89,96498.0,17445.0,89367.0,-2.35,22.07,92.61,135.12,29.0,0.20,0.82,11.28,XR,NTT,PHARMA


In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,5664.39,7262.39,16.31,X-LC,53.93,183998.0,14066.0,33874.0,5.01,8.28,18.41,28.21,0.0,0.42,1.57,35.16,X200,ATH,IT
29,GICRE,335.06,583.00,13.56,M-MC,97.75,194836.0,48415.0,59932.0,-4.10,33.07,30.76,74.00,137.0,0.81,1.66,50.60,XY24,BTT,INSURANCE
41,INFY,1436.24,2275.00,9.90,X-LC,88.76,204849.0,55480.0,31752.0,2.73,37.14,15.50,58.40,37.0,1.75,1.75,42.94,X40,BTT,IT
70,SFL,1034.30,1287.00,29.04,M-SC,96.63,239889.0,-12480.0,74150.0,-1.75,-4.95,30.91,24.43,-7.0,-0.17,2.04,26.04,XY24,NTT,MISC


In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-132.42,L-SC,100.00,58138.0,-55411.0,95463.0,0.00,-48.80,164.20,35.27,NaN,-0.58,0.50,33.86,XR,NTT,HOTELS
10,BAJFINANCE,7055.29,9150.00,-27.76,X-LC,98.88,226354.0,7640.0,57290.0,0.33,3.49,25.31,29.69,12.0,0.13,1.93,16.27,X40,BTT,FINANCE
29,GICRE,335.06,583.00,13.56,M-MC,97.75,194836.0,48415.0,59932.0,-4.10,33.07,30.76,74.00,137.0,0.81,1.66,50.60,XY24,BTT,INSURANCE
70,SFL,1034.30,1287.00,29.04,M-SC,96.63,239889.0,-12480.0,74150.0,-1.75,-4.95,30.91,24.43,-7.0,-0.17,2.04,26.04,XY24,NTT,MISC
17,BRITANNIA,4845.40,6446.05,0.87,X-LC,95.51,108647.0,2048.0,33170.0,0.25,1.92,30.53,33.03,239.0,0.06,0.93,7.16,AR,ATH,FMCG


In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.51
1,25,40.78
2,50,70.79


In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    40.16
X40     21.64
X40N    14.64
XR      14.46
AR       6.15
X200     1.57
SR       1.40
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
M-SC    28.19
X-LC    21.72
H-SC    17.67
M-MC    10.92
X-SC     5.74
X-MC     4.12
M-LC     3.34
H-MC     2.71
L-SC     2.66
H-LC     1.37
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,14.04,-15.90,70.77
FINANCE,12.37,-12.51,59.89
IT,8.94,-6.12,59.46
FMCG,7.92,-2.11,40.32
MISC,7.43,-3.92,42.44
PAINTS,5.93,-18.11,53.06
CHEMICALS,5.78,-6.86,74.28
HEALTHCARE,4.93,5.69,40.98
INSURANCE,4.36,5.48,45.51


In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2944158.0,30
XR,1322018.0,18
X40N,1016274.0,17
X40,842893.0,14
AR,632055.0,7
SR,181074.0,2
X200,33874.0,1


In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
M-SC,2202911.0,28
H-SC,1682727.0,16
M-MC,910779.0,10
X-LC,783295.0,14
X-SC,437468.0,6
L-SC,286607.0,4
X-MC,196441.0,4
H-MC,149498.0,2
M-LC,130393.0,3


In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M-SC       XY24      1031485.0     11
M-MC       XY24       702923.0      7
X-LC       X40        665807.0     11
H-SC       XY24       643505.0      4
M-SC       X40N       479787.0      7
           XR         453098.0      6
H-SC       XR         383065.0      5
           X40N       361825.0      6
           AR         294332.0      1
X-SC       XR         173481.0      2
L-SC       XR         159421.0      2
H-MC       XY24       149498.0      2
M-MC       XR         131114.0      2
M-SC       AR         123412.0      2
X-SC       SR         120149.0      1
M-LC       XY24       108554.0      2
X-MC       X40N        93247.0      2
X-LC       AR          83614.0      2
X-SC       X40N        81415.0      2
M-MC       AR          76742.0      1
H-LC       X40         74815.0      1
L-SC       XY24        73231.0      1
X-SC       XY24        62423.0      1
M-SC       SR          60925.0      1
X-MC       XY24        55127.0      1
M-SC       X40         54204.0      1
L-SC       AR          53955.0      1
X-MC       X40         48067.0      1
X-LC       X200        33874.0      1
M-LC       XR          21839.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 22.0 seconds
